<a href="https://colab.research.google.com/github/arminarj/convex-optimization-boyd/blob/master/Online_advertising_displays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install cvxpy==1.1.0a4 -q
!pip install diffcp -q

     |████████████████████████████████| 993kB 9.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.2MB 96kB/s 


Part A:

In [0]:
import cvxpy as cvx
import numpy as np

np.random.seed(0)
n = 100   #number of ads
m = 30    #number of contracts
T = 60    #number of periods

#number of impressions in each period
I = 10*np.random.rand(T, 1); I = np.asmatrix(I)
#revenue rate for each period and ad
R = np.random.rand(n, T); R = np.asmatrix(R)
#contract target number of impressions
q = T/float(n)*50*np.random.rand(m, 1); q = np.asmatrix(q)
#penalty rate for shortfall
p = np.random.rand(m, 1); p = np.asmatrix(p)
#one column per contract. 1's at the periods to be displayed
Tcontr = np.matrix(np.random.rand(T, m)>.8, dtype = float)
Acontr = np.zeros((n, m)); Acont = np.asmatrix(Acontr)
for i in range(n):
    contract=int(np.floor(m*np.random.rand(1)))
    #one column per contract. 1's at the ads to be displayed
    Acontr[i,contract]=1


In [0]:
import cvxpy as cp

N = cp.Variable((T, n))

N_sum = cp.diag(Tcontr.T @ N @ Acontr)
S = cp.pos( q[0] - N_sum)
penalty = S @ p

revenue = cp.sum(cp.multiply(R ,N.T ))

obj_function =  revenue - penalty

obj = cp.Maximize(obj_function)

constraints  = [
                0 <= N,
                cp.sum(N, axis=1) == I[0,:],
]

prob = cp.Problem(obj, constraints)

print("prob is DCP:", prob.is_dcp())

try:
    prob.solve(requires_grad=True)
    print("status:  ", prob.status)
    print("optimal value:   ", prob.value)
    print("revenue values:  ", revenue.value)
    print("Penalty values:  ", penalty.value)

except Exception as e:
    print(e)


prob is DCP: True
status:   optimal
optimal value:    135.7002514948442
revenue values:   282.85717086477376
Penalty values:   [[147.15928987]]
